In [1]:
taxi = pd.read_csv("/Users/jisuim/Desktop/tp_taxi/jisu/train.csv")

In [2]:
from sklearn.tree import DecisionTreeRegressor

In [3]:
taxi_new = taxi.copy()

# Feature Engineering

In [4]:
from math import cos, asin, sqrt
def dist(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295     #Pi/180
    a = 0.5 - cos((lat2 - lat1) * p)/2 + cos(lat1 * p) * cos(lat2 * p) * (1 - cos((lon2 - lon1) * p)) / 2
    return 12742 * asin(sqrt(a))

In [5]:
for i in range(len(taxi)):
    lat1 = taxi['pickup_latitude'][i]
    lon1 = taxi['pickup_longitude'][i]
    lat2 = taxi['dropoff_latitude'][i]
    lon2 = taxi['dropoff_longitude'][i]
    taxi_new.at[i, 'distance'] = dist(lat1, lon1, lat2, lon2)


In [7]:
Morning = ['01','02','03','04','05','06','07']
Night = ['19', '20', '21', '22', '23', '00']
for i in range(len(taxi_new)):
    k = taxi['pickup_datetime'][i][11:13]
    if k in Morning:
        taxi_new.at[i, "time_n"] = "Morning"
    elif k in Night:
        taxi_new.at[i, "time_n"] = "Night"
    else:
        taxi_new.at[i, "time_n"] = "Duty"
        
     

In [162]:
taxi_new.loc[taxi_new.distance > 200] = np.nan ##200km 넘는 데이터 제거
taxi_new.loc[taxi_new.trip_duration > 30000] = np.nan ##40000초(약 11시간)가 넘는 데이터 제거
taxi_new.loc[taxi_new.passenger_count == 0] = np.NAN   ### passenger 수가 0인 데이터 제거
taxi_new.dropna(inplace=True)

In [163]:
dummy_time_n = pd.get_dummies(taxi_new['time_n'],prefix='time_n')
dummy_vendor_id = pd.get_dummies(taxi_new['vendor_id'], prefix='id')
dummy_store = pd.get_dummies(taxi_new['store_and_fwd_flag'], prefix='snf')

In [12]:
len(taxi_new)

1456542

In [13]:
len(taxi)

1458644

In [14]:
taxi_new.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,distance,time_n
0,id2875421,2.0,2016-03-14 17:24:55,2016-03-14 17:32:30,1.0,-73.982155,40.767937,-73.964630,40.765602,N,455.0,1.498521,Duty
1,id2377394,1.0,2016-06-12 00:43:35,2016-06-12 00:54:38,1.0,-73.980415,40.738564,-73.999481,40.731152,N,663.0,1.805507,Night
2,id3858529,2.0,2016-01-19 11:35:24,2016-01-19 12:10:48,1.0,-73.979027,40.763939,-74.005333,40.710087,N,2124.0,6.385098,Duty
3,id3504673,2.0,2016-04-06 19:32:31,2016-04-06 19:39:40,1.0,-74.010040,40.719971,-74.012268,40.706718,N,429.0,1.485498,Night
4,id2181028,2.0,2016-03-26 13:30:55,2016-03-26 13:38:10,1.0,-73.973053,40.793209,-73.972923,40.782520,N,435.0,1.188588,Duty


> passenger_count, 출발위치, 도착위치, 시간대, 직선거리, vendor_id, store_and_fwd_flag

# Decision Tree

In [165]:
X = taxi_new[['ave_speed_x', 'ave_speed_y', 'distance']]

In [166]:
X.head()

,ave_speed_x,ave_speed_y,distance
0,12.946163,13.731112,1.498521
1,13.838090,12.750622,1.805507
2,12.260338,15.753882,6.385098
3,14.736821,15.753882,1.485498
4,15.942333,14.571577,1.188588


In [167]:
dfX = pd.concat([X, dummy_store, dummy_time_n, dummy_vendor_id], axis=1)

In [168]:
len(dfX)

1456542

In [153]:
dfX.head()

,ave_speed_x,ave_speed_y,distance,snf_N,snf_Y,time_n_Duty,time_n_Morning,time_n_Night,id_1.0,id_2.0
0,12.946163,13.731112,1.498521,1.0,0.0,1.0,0.0,0.0,0.0,1.0
1,13.838090,12.750622,1.805507,1.0,0.0,0.0,0.0,1.0,1.0,0.0
2,12.260338,15.753882,6.385098,1.0,0.0,1.0,0.0,0.0,0.0,1.0
3,14.736821,15.753882,1.485498,1.0,0.0,0.0,0.0,1.0,0.0,1.0
4,15.942333,14.571577,1.188588,1.0,0.0,1.0,0.0,0.0,0.0,1.0


In [169]:
y = taxi_new['trip_duration']

In [170]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dfX, y, test_size=0.25)

In [171]:
tree = DecisionTreeRegressor(max_depth=7, random_state=0)
tree.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=7, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=0, splitter='best')

In [172]:
tree.score(X_train, y_train)

0.6665426892396675

In [173]:
tree.score(X_test, y_test)

0.6632770536954776

In [174]:
tree.feature_importances_

array([2.58611313e-03, 2.59438287e-02, 9.02647880e-01, 0.00000000e+00,
       0.00000000e+00, 6.10924009e-02, 4.27664041e-03, 3.21089112e-03,
       2.42246243e-04, 0.00000000e+00])

In [175]:
tree_2 = DecisionTreeRegressor(max_depth=9, random_state=1)
tree_2.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=9, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=1, splitter='best')

In [176]:
tree_2.score(X_train, y_train)

0.6821899430164705

In [177]:
tree_2.score(X_test, y_test)

0.6693884666759853

In [181]:
tree_3 = DecisionTreeRegressor(max_depth=12, random_state=1)
tree_3.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=12, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=1, splitter='best')

In [182]:
tree_3.score(X_train, y_train)

0.7088648184285258

In [183]:
tree_3.score(X_test, y_test)

0.6675879114347754

# test.csv

In [37]:
taxi_test = pd.read_csv("/Users/jisuim/Desktop/tp_taxi/jisu/test.csv")

In [38]:
for i in range(len(taxi_test)):
    lat1 = taxi_test['pickup_latitude'][i]
    lon1 = taxi_test['pickup_longitude'][i]
    lat2 = taxi_test['dropoff_latitude'][i]
    lon2 = taxi_test['dropoff_longitude'][i]
    taxi_test.at[i, 'distance'] = dist(lat1, lon1, lat2, lon2)


In [39]:
### 시간대 19~00(Night) / 01~07(Morning) / 08~18(Duty) 로 분할하기
Morning = ['01','02','03','04','05','06','07']
Night = ['19', '20', '21', '22', '23', '00']
for i in range(len(taxi_test)):
    k = taxi_test['pickup_datetime'][i][11:13]
    if k in Morning:
        taxi_test.at[i, "time_n"] = "Morning"
    elif k in Night:
        taxi_test.at[i, "time_n"] = "Night"
    else:
        taxi_test.at[i, "time_n"] = "Duty"
        
     

In [185]:
tdummy_time_n = pd.get_dummies(taxi_test['time_n'],prefix='time_n')
tdummy_vendor_id = pd.get_dummies(taxi_test['vendor_id'], prefix='id')
tdummy_store = pd.get_dummies(taxi_test['store_and_fwd_flag'], prefix='snf')

In [184]:
X_t = taxi_test[['ave_speed_x','ave_speed_y', 'distance']]

In [186]:
dfX_t = pd.concat([X_t, tdummy_store, tdummy_time_n, tdummy_vendor_id], axis=1)

In [43]:
t_y = tree.predict(dfX_t)

In [47]:
t_y = pd.DataFrame(t_y)

In [48]:
t_y.head()

,0
0,782.666574
1,782.666574
2,470.893991
3,1189.425024
4,388.223893


In [49]:
submission_test = pd.concat([taxi_test['id'], t_y], axis=1)

In [50]:
submission_test.to_csv('submission_2.csv', sep=',', na_rep='NaN')

In [58]:
t_y_2 = tree_2.predict(dfX_t)

In [59]:
t_y_2 = pd.DataFrame(t_y_2)

In [60]:
t_y_2.head()

,0
0,818.761064
1,722.254658
2,449.997410
3,1245.218229
4,396.469859


In [61]:
submission_test = pd.concat([taxi_test['id'], t_y_2], axis=1)

In [62]:
submission_test.to_csv('submission_3.csv', sep=',', na_rep='NaN')

In [90]:
t_y_3 = tree_3.predict(dfX_t)

In [93]:
t_y_3 = pd.DataFrame(t_y_3)

In [94]:
submission_test = pd.concat([taxi_test['id'], t_y_3], axis=1)

In [95]:
submission_test.to_csv('submission_4.csv', sep=',', na_rep='NaN')

In [192]:
dfX_t = dfX_t.fillna(value=0)
dfX_t = pd.DataFrame(dfX_t)

In [194]:
### speed 추가, depth = 12
t_y_4 = tree_3.predict(dfX_t)
t_y_4 = pd.DataFrame(t_y_4)
submission_test = pd.concat([taxi_test['id'], t_y_4], axis=1)
submission_test.to_csv('submission_6.csv', sep=',', na_rep='NaN')

# Gradient Boosting

In [195]:
from sklearn.ensemble import GradientBoostingRegressor

In [199]:
gbr = GradientBoostingRegressor(learning_rate=0.1, n_estimators= 20, random_state=0, max_depth=11)
gbr.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=11, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=20, presort='auto', random_state=0,
             subsample=1.0, verbose=0, warm_start=False)

In [197]:
gbr.score(X_train, y_train) # n_estimators = 20, max_depth = 9

0.6884425292508188

In [198]:
gbr.score(X_test, y_test) # n_estimators = 20, max_depth = 9

0.6723305867581697

In [201]:
gbr.score(X_train, y_train) # n_estimators = 20, max_depth = 11

0.7151521935072287

In [109]:
t_y_g = gbr.predict(dfX_t)

In [111]:
t_y_g = pd.DataFrame(t_y_g)

In [112]:
submission_test = pd.concat([taxi_test['id'], t_y_g], axis=1)
submission_test.to_csv('submission_5.csv', sep=',', na_rep='NaN')

In [113]:
t_y_g.head()

,0
0,832.122566
1,700.280096
2,545.373741
3,1073.457954
4,434.638600


In [200]:
t_y_g2 = gbr.predict(dfX_t)     ## n_estimators=20, max_depth=11
t_y_g2 = pd.DataFrame(t_y_g2)

In [202]:
submission_test = pd.concat([taxi_test['id'], t_y_g2], axis=1)
submission_test.to_csv('submission_7.csv', sep=',', na_rep='NaN')

# New Feature -> Speed

> 1도 차이 : 약 133km

> 0.01도 차이 : 약 1.3km

> 0.001도 차이 : 약 130m

In [115]:
taxi_new["p_lon"] = np.round(taxi_new['pickup_longitude'], 2)
taxi_new["p_lat"] = np.round(taxi_new['pickup_latitude'], 2)
taxi_new["d_lon"] = np.round(taxi_new['dropoff_longitude'], 2)
taxi_new["d_lat"] = np.round(taxi_new['dropoff_latitude'], 2)

In [132]:
taxi_new.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,distance,time_n,p_lon,p_lat,d_lon,d_lat
0,id2875421,2.0,2016-03-14 17:24:55,2016-03-14 17:32:30,1.0,-73.982155,40.767937,-73.964630,40.765602,N,455.0,1.498521,Duty,-73.98,40.77,-73.96,40.77
1,id2377394,1.0,2016-06-12 00:43:35,2016-06-12 00:54:38,1.0,-73.980415,40.738564,-73.999481,40.731152,N,663.0,1.805507,Night,-73.98,40.74,-74.00,40.73
2,id3858529,2.0,2016-01-19 11:35:24,2016-01-19 12:10:48,1.0,-73.979027,40.763939,-74.005333,40.710087,N,2124.0,6.385098,Duty,-73.98,40.76,-74.01,40.71
3,id3504673,2.0,2016-04-06 19:32:31,2016-04-06 19:39:40,1.0,-74.010040,40.719971,-74.012268,40.706718,N,429.0,1.485498,Night,-74.01,40.72,-74.01,40.71
4,id2181028,2.0,2016-03-26 13:30:55,2016-03-26 13:38:10,1.0,-73.973053,40.793209,-73.972923,40.782520,N,435.0,1.188588,Duty,-73.97,40.79,-73.97,40.78


In [133]:
taxi_new["speed"] = taxi_new["distance"] / taxi_new['trip_duration'] * 3600 ## 속도 (km/hour)

In [134]:
taxi_new.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,distance,time_n,p_lon,p_lat,d_lon,d_lat,speed
0,id2875421,2.0,2016-03-14 17:24:55,2016-03-14 17:32:30,1.0,-73.982155,40.767937,-73.964630,40.765602,N,455.0,1.498521,Duty,-73.98,40.77,-73.96,40.77,11.856428
1,id2377394,1.0,2016-06-12 00:43:35,2016-06-12 00:54:38,1.0,-73.980415,40.738564,-73.999481,40.731152,N,663.0,1.805507,Night,-73.98,40.74,-74.00,40.73,9.803659
2,id3858529,2.0,2016-01-19 11:35:24,2016-01-19 12:10:48,1.0,-73.979027,40.763939,-74.005333,40.710087,N,2124.0,6.385098,Duty,-73.98,40.76,-74.01,40.71,10.822201
3,id3504673,2.0,2016-04-06 19:32:31,2016-04-06 19:39:40,1.0,-74.010040,40.719971,-74.012268,40.706718,N,429.0,1.485498,Night,-74.01,40.72,-74.01,40.71,12.465721
4,id2181028,2.0,2016-03-26 13:30:55,2016-03-26 13:38:10,1.0,-73.973053,40.793209,-73.972923,40.782520,N,435.0,1.188588,Duty,-73.97,40.79,-73.97,40.78,9.836594


In [135]:
taxi_new_p = taxi_new[['speed', 'p_lon', 'p_lat']].groupby(['p_lon', 'p_lat']).mean().reset_index()
taxi_new_p = taxi_new_p.rename(columns = {'speed': 'ave_speed'})
taxi_new_d = taxi_new[['speed', 'd_lon', 'd_lat']].groupby(['d_lon', 'd_lat']).mean().reset_index()
taxi_new_d = taxi_new_d.rename(columns = {'speed': 'ave_speed'})

In [136]:
taxi_new = pd.merge(taxi_new, taxi_new_p, on = ['p_lon', 'p_lat'], how = 'left')

In [137]:
taxi_new.head(3)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,distance,time_n,p_lon,p_lat,d_lon,d_lat,speed,ave_speed
0,id2875421,2.0,2016-03-14 17:24:55,2016-03-14 17:32:30,1.0,-73.982155,40.767937,-73.964630,40.765602,N,455.0,1.498521,Duty,-73.98,40.77,-73.96,40.77,11.856428,12.946163
1,id2377394,1.0,2016-06-12 00:43:35,2016-06-12 00:54:38,1.0,-73.980415,40.738564,-73.999481,40.731152,N,663.0,1.805507,Night,-73.98,40.74,-74.00,40.73,9.803659,13.838090
2,id3858529,2.0,2016-01-19 11:35:24,2016-01-19 12:10:48,1.0,-73.979027,40.763939,-74.005333,40.710087,N,2124.0,6.385098,Duty,-73.98,40.76,-74.01,40.71,10.822201,12.260338


In [138]:
taxi_new = pd.merge(taxi_new, taxi_new_d, on = ['d_lon', 'd_lat'], how = 'left')

In [139]:
taxi_new.head(3)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,distance,time_n,p_lon,p_lat,d_lon,d_lat,speed,ave_speed_x,ave_speed_y
0,id2875421,2.0,2016-03-14 17:24:55,2016-03-14 17:32:30,1.0,-73.982155,40.767937,-73.964630,40.765602,N,455.0,1.498521,Duty,-73.98,40.77,-73.96,40.77,11.856428,12.946163,13.731112
1,id2377394,1.0,2016-06-12 00:43:35,2016-06-12 00:54:38,1.0,-73.980415,40.738564,-73.999481,40.731152,N,663.0,1.805507,Night,-73.98,40.74,-74.00,40.73,9.803659,13.838090,12.750622
2,id3858529,2.0,2016-01-19 11:35:24,2016-01-19 12:10:48,1.0,-73.979027,40.763939,-74.005333,40.710087,N,2124.0,6.385098,Duty,-73.98,40.76,-74.01,40.71,10.822201,12.260338,15.753882


In [142]:

taxi_test["p_lon"] = np.round(taxi_test['pickup_longitude'], 2)
taxi_test["p_lat"] = np.round(taxi_test['pickup_latitude'], 2)
taxi_test["d_lon"] = np.round(taxi_test['dropoff_longitude'], 2)
taxi_test["d_lat"] = np.round(taxi_test['dropoff_latitude'], 2)

In [143]:
taxi_test = pd.merge(taxi_test, taxi_new_p, on = ['p_lon', 'p_lat'], how = 'left')

In [148]:
taxi_test = pd.merge(taxi_test, taxi_new_d, on = ['d_lon', 'd_lat'], how = 'left')

In [149]:
taxi_test.head()

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,distance,time_n,p_lon,p_lat,d_lon,d_lat,ave_speed_x,ave_speed_y
0,id3004672,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,N,2.746426,Night,-73.99,40.73,-73.99,40.76,13.382748,11.899024
1,id3505355,1,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,N,2.759239,Night,-73.96,40.68,-73.96,40.66,16.003539,19.886573
2,id1217141,1,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,N,1.306155,Night,-74.00,40.74,-73.99,40.73,13.431413,12.883836
3,id2150126,2,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469,N,5.269088,Night,-73.96,40.77,-73.99,40.73,13.867467,12.883836
4,id1598245,1,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890,N,0.960842,Night,-73.97,40.76,-73.96,40.76,13.588606,13.193883
